# 02.5 - Master Races Verification

This notebook performs comprehensive completeness and quality verification on the combined `master_races.csv` dataset.

**Purpose**: Verify that the data combining process created a complete and correct master dataset.

**What this notebook checks:**
1. **Basic Structure**: Rows, columns, data types
2. **Uniqueness**: One row per (raceId, driverId) combination
3. **Temporal Coverage**: Year and race coverage (1994-2024)
4. **Merge Success Rates**: How well each data source merged
5. **Completeness**: Missing values by column and source
6. **Data Quality**: Value ranges, relationships, consistency
7. **Target Variable**: Podium distribution and patterns

**When to use**: After running `02_data_combining.ipynb` to verify the combined dataset is ready for EDA and modeling.


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set up paths
PROJECT_ROOT = Path(r"C:\Users\erikv\Downloads\F1")
PROCESSED_ROOT = PROJECT_ROOT / "data" / "processed"
KAGGLE_ROOT = PROJECT_ROOT / "data" / "raw" / "kaggle"

print(f"Processed data: {PROCESSED_ROOT}")
print(f"Kaggle data (for reference): {KAGGLE_ROOT}")


Processed data: C:\Users\erikv\Downloads\F1\data\processed
Kaggle data (for reference): C:\Users\erikv\Downloads\F1\data\raw\kaggle


## 1. Load Master Races Dataset

Load the combined master_races.csv file and display basic information.


In [2]:
# Load master_races.csv
master_path = PROCESSED_ROOT / "master_races.csv"

if not master_path.exists():
    raise FileNotFoundError(f"master_races.csv not found at {master_path}. Run 02_data_combining.ipynb first.")

master = pd.read_csv(master_path, low_memory=False)

# Convert date to datetime
if 'date' in master.columns:
    master['date'] = pd.to_datetime(master['date'], errors='coerce')

print(f"Master Races Dataset Loaded")
print(f"=" * 60)
print(f"Shape: {master.shape[0]:,} rows × {master.shape[1]} columns")
print(f"Date range: {master['date'].min()} to {master['date'].max()}")
print(f"Years: {master['year'].min()} - {master['year'].max()}")
print(f"\nFirst few rows:")
master.head(3)


Master Races Dataset Loaded
Shape: 12,358 rows × 82 columns
Date range: 1994-03-27 00:00:00 to 2024-12-08 00:00:00
Years: 1994 - 2024

First few rows:


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,sprint_results_time,sprint_results_milliseconds,sprint_results_fastestLap,sprint_results_fastestLapTime,sprint_results_statusId,lap_time_variance,throttle_variance,overtake_attempts,avg_pit_stops,podium
0,1,18,1,1,22,1,1,1,1,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,18,2,2,3,5,2,2,2,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,3,18,3,3,7,7,3,3,3,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


**Expected Output:**
- **Rows**: Should be ~12,000-13,000 (one per raceId+driverId for 1994+)
- **Columns**: Should be ~80-82 (combined from all sources)
- **Date range**: 1994-03-27 to 2024-12-08 (or latest race date)
- **Years**: 1994 - 2024

**What to look for:**
- ✅ **Good signs**: 
  - Expected number of rows (~12K)
  - Date range starts from 1994
  - All expected columns present
- ⚠️ **Warning signs**: 
  - Much fewer rows than expected (missing data)
  - Date range doesn't start in 1994 (filter didn't work)
  - Missing expected columns


## 2. Uniqueness Verification

Verify that each (raceId, driverId) combination appears exactly once.


In [ ]:
# Check for duplicate (raceId, driverId) combinations
duplicate_check = master.groupby(['raceId', 'driverId']).size()
duplicates = duplicate_check[duplicate_check > 1]

if len(duplicates) == 0:
    print("✅ PASS: No duplicate (raceId, driverId) combinations found")
    print(f"   Each driver-race combination appears exactly once.")
else:
    print(f"⚠️ WARNING: Found {len(duplicates)} duplicate (raceId, driverId) combinations:")
    print(duplicates.head(10))
    print(f"\nFirst duplicate example:")
    example_race = duplicates.index[0][0]
    example_driver = duplicates.index[0][1]
    print(master[(master['raceId'] == example_race) & (master['driverId'] == example_driver)])

print(f"\nTotal unique (raceId, driverId) combinations: {len(duplicate_check):,}")
print(f"Total rows: {len(master):,}")
print(f"Match: {len(duplicate_check) == len(master)}")


**Expected Output:**
- **No duplicates**: Each (raceId, driverId) should appear exactly once
- **Match**: Number of unique combinations should equal total rows

**What to look for:**
- ✅ **Good signs**: No duplicates found, counts match
- ⚠️ **Warning signs**: Duplicates indicate merge issues or data quality problems


## 3. Temporal Coverage Analysis

Check year and race coverage to ensure complete temporal data.


In [ ]:
# Year coverage
year_coverage = master.groupby('year').agg({
    'raceId': 'nunique',
    'driverId': 'nunique',
    'resultId': 'count'
}).rename(columns={
    'raceId': 'unique_races',
    'driverId': 'unique_drivers',
    'resultId': 'total_results'
})

print("Year Coverage Analysis:")
print("=" * 60)
print(year_coverage)

# Check for missing years in 1994-2024 range
expected_years = set(range(1994, 2025))
actual_years = set(master['year'].unique())
missing_years = expected_years - actual_years

if missing_years:
    print(f"\n⚠️ WARNING: Missing years: {sorted(missing_years)}")
else:
    print(f"\n✅ All expected years present (1994-2024)")

# Race coverage per year
print(f"\nRace Coverage Summary:")
print(f"  Average races per year: {year_coverage['unique_races'].mean():.1f}")
print(f"  Min races in a year: {year_coverage['unique_races'].min()}")
print(f"  Max races in a year: {year_coverage['unique_races'].max()}")
print(f"  Typical F1 season has 20-24 races")


## 4. Merge Success Rates

Check how successfully each data source merged with the base results table.


In [ ]:
# Define key indicator columns for each merged source
merge_indicators = {
    'results': ['resultId'],  # Base table - should be 100%
    'races': ['year', 'date'],  # Should be 100% (used for filtering)
    'circuits': ['circuit_name', 'country'],
    'drivers': ['forename', 'surname'],
    'constructors': ['name_constructor'],
    'driver_standings': ['driverStandingsId', 'driver_standings_points'],
    'constructor_standings': ['constructorStandingsId', 'constructor_standings_points'],
    'constructor_results': ['constructorResultsId', 'constructor_results_points'],
    'qualifying': ['qualifyId', 'q1'],
    'sprint_results': ['sprint_results_resultId']
}

print("Merge Success Rates:")
print("=" * 60)

merge_stats = []
for source, indicator_cols in merge_indicators.items():
    # Check if at least one indicator column exists and has data
    found_cols = [col for col in indicator_cols if col in master.columns]
    
    if not found_cols:
        coverage_pct = 0.0
        status = "❌ NOT MERGED"
    else:
        # Use the first found column for coverage calculation
        coverage_col = found_cols[0]
        coverage_count = master[coverage_col].notna().sum()
        coverage_pct = (coverage_count / len(master)) * 100
        status = "✅" if coverage_pct > 95 else "⚠️" if coverage_pct > 50 else "❌"
    
    merge_stats.append({
        'source': source,
        'coverage_pct': coverage_pct,
        'coverage_count': coverage_count if found_cols else 0,
        'status': status
    })
    
    print(f"{status} {source:25s}: {coverage_pct:6.2f}% ({coverage_count:,}/{len(master):,} rows)")

# Summary
merge_df = pd.DataFrame(merge_stats)
print(f"\nSummary:")
print(f"  Fully merged (>95%): {(merge_df['coverage_pct'] > 95).sum()} sources")
print(f"  Partially merged (50-95%): {((merge_df['coverage_pct'] >= 50) & (merge_df['coverage_pct'] <= 95)).sum()} sources")
print(f"  Poorly merged (<50%): {(merge_df['coverage_pct'] < 50).sum()} sources")


## 5. Missing Values Analysis

Analyze missing values by column and data source to identify data gaps.


In [ ]:
# Calculate missing values
missing_stats = []
for col in master.columns:
    missing_count = master[col].isnull().sum()
    missing_pct = (missing_count / len(master)) * 100
    missing_stats.append({
        'column': col,
        'missing_count': missing_count,
        'missing_pct': missing_pct,
        'dtype': str(master[col].dtype)
    })

missing_df = pd.DataFrame(missing_stats).sort_values('missing_pct', ascending=False)

# Group by expected missing patterns
print("Missing Values Analysis:")
print("=" * 60)

# Placeholder columns (expected to be 100% missing)
placeholder_cols = ['lap_time_variance', 'throttle_variance', 'overtake_attempts', 'avg_pit_stops']
print(f"\n1. Placeholder Features (expected 100% missing):")
placeholder_missing = missing_df[missing_df['column'].isin(placeholder_cols)]
print(placeholder_missing[['column', 'missing_pct']].to_string(index=False))

# Sprint results (expected high missing - only recent seasons)
sprint_cols = [col for col in master.columns if col.startswith('sprint_results_')]
print(f"\n2. Sprint Results (expected ~97% missing - only 2021+):")
sprint_missing = missing_df[missing_df['column'].isin(sprint_cols)].head(5)
print(sprint_missing[['column', 'missing_pct']].to_string(index=False))

# Other high missing columns
print(f"\n3. Other Columns with Missing Values (>5%):")
other_missing = missing_df[
    (~missing_df['column'].isin(placeholder_cols + sprint_cols)) & 
    (missing_df['missing_pct'] > 5)
]
if len(other_missing) > 0:
    print(other_missing[['column', 'missing_pct']].to_string(index=False))
else:
    print("  None - all non-placeholder columns have <5% missing")

# Critical columns (should have minimal missing)
critical_cols = ['raceId', 'driverId', 'year', 'date', 'positionOrder', 'points', 'circuit_name', 'forename', 'surname']
print(f"\n4. Critical Columns Missing Check:")
critical_missing = missing_df[missing_df['column'].isin(critical_cols)]
for _, row in critical_missing.iterrows():
    status = "✅" if row['missing_pct'] == 0 else "⚠️" if row['missing_pct'] < 1 else "❌"
    print(f"  {status} {row['column']:25s}: {row['missing_pct']:6.2f}% missing")


**Expected Output:**
- **Placeholder features**: 100% missing (expected - will be filled later)
- **Sprint results**: ~97% missing (only 2021+ seasons have sprints)
- **Qualifying (q2, q3)**: ~20-40% missing (not all drivers make it to Q2/Q3)
- **Critical columns**: 0% missing (raceId, driverId, year, date, positionOrder, etc.)

**What to look for:**
- ✅ **Good signs**: 
  - Critical columns have 0% missing
  - Expected patterns (sprints, placeholders)
- ⚠️ **Warning signs**: 
  - Critical columns with missing values (data quality issue)
  - Unexpected high missing rates in non-placeholder columns


## 6. Data Quality Checks

Verify value ranges, relationships, and consistency across columns.


In [ ]:
print("Data Quality Checks:")
print("=" * 60)

# 1. Position validation
print("\n1. Position Validation:")
print(f"  positionOrder range: {master['positionOrder'].min()} - {master['positionOrder'].max()}")
print(f"  positionOrder > 20: {(master['positionOrder'] > 20).sum():,} rows ({((master['positionOrder'] > 20).sum() / len(master) * 100):.1f}%)")
print(f"  positionOrder <= 0: {(master['positionOrder'] <= 0).sum():,} rows")

# Check if points match position (top 10 get points in most eras)
top_positions = master[master['positionOrder'] <= 10]
points_in_top10 = (top_positions['points'] > 0).sum()
print(f"  Rows with positionOrder <= 10 and points > 0: {points_in_top10:,}/{len(top_positions):,} ({points_in_top10/len(top_positions)*100:.1f}%)")

# 2. Date consistency
print("\n2. Date Consistency:")
print(f"  Invalid dates: {master['date'].isnull().sum()} rows")
print(f"  Dates outside 1994-2024: {((master['date'] < '1994-01-01') | (master['date'] > '2024-12-31')).sum()} rows")

# 3. Year consistency
print("\n3. Year Consistency:")
print(f"  Years < 1994: {(master['year'] < 1994).sum()} rows")
print(f"  Years > 2024: {(master['year'] > 2024).sum()} rows")
year_date_match = (master['date'].dt.year == master['year']).sum()
print(f"  Year matches date year: {year_date_match:,}/{len(master):,} ({year_date_match/len(master)*100:.1f}%)")

# 4. Driver-Constructor relationship
print("\n4. Driver-Constructor Consistency:")
print(f"  Unique drivers: {master['driverId'].nunique():,}")
print(f"  Unique constructors: {master['constructorId'].nunique():,}")
print(f"  Unique (driverId, constructorId) pairs: {master[['driverId', 'constructorId']].drop_duplicates().shape[0]:,}")

# 5. Podium target variable
print("\n5. Target Variable (Podium) Validation:")
if 'podium' in master.columns:
    podium_dist = master['podium'].value_counts()
    podium_rate = master['podium'].mean()
    print(f"  Podium (1): {podium_dist.get(1, 0):,} rows ({podium_rate*100:.2f}%)")
    print(f"  Non-podium (0): {podium_dist.get(0, 0):,} rows ({(1-podium_rate)*100:.2f}%)")
    print(f"  Expected podium rate: ~14-15% (3 podiums per ~20 drivers)")
    
    # Check if podium matches positionOrder
    podium_matches = (master['podium'] == (master['positionOrder'] <= 3).astype(int)).sum()
    print(f"  Podium matches positionOrder <= 3: {podium_matches:,}/{len(master):,} ({podium_matches/len(master)*100:.1f}%)")


**Expected Output:**
- **positionOrder**: Range from 1 to ~22-24 (max grid positions)
- **Points**: Most positions <= 10 should have points > 0
- **Dates**: All valid, within 1994-2024 range
- **Year consistency**: Year should match date.year
- **Podium rate**: ~14-15% (3 podiums per ~20 drivers per race)
- **Podium definition**: Should match positionOrder <= 3

**What to look for:**
- ✅ **Good signs**: 
  - Reasonable position ranges
  - Dates are valid and consistent
  - Podium rate around 14-15%
  - Podium matches positionOrder definition
- ⚠️ **Warning signs**: 
  - Invalid positions (< 1 or unreasonably high)
  - Invalid dates or year mismatches
  - Podium rate far from expected (~14-15%)


## 7. Coverage by Data Source (Detailed)

Check coverage for each merged source in more detail, especially for time-sensitive data.


In [ ]:
# Check qualifying coverage by year
if 'qualifyId' in master.columns:
    print("Qualifying Coverage by Year:")
    print("=" * 60)
    qualifying_by_year = master.groupby('year').agg({
        'qualifyId': lambda x: x.notna().sum(),
        'resultId': 'count'
    })
    qualifying_by_year['coverage_pct'] = (qualifying_by_year['qualifyId'] / qualifying_by_year['resultId'] * 100)
    qualifying_by_year.columns = ['with_qualifying', 'total_results', 'coverage_pct']
    print(qualifying_by_year)

# Check sprint results coverage by year (should only be 2021+)
if 'sprint_results_resultId' in master.columns:
    print("\n\nSprint Results Coverage by Year:")
    print("=" * 60)
    sprint_by_year = master.groupby('year').agg({
        'sprint_results_resultId': lambda x: x.notna().sum(),
        'resultId': 'count'
    })
    sprint_by_year['coverage_pct'] = (sprint_by_year['sprint_results_resultId'] / sprint_by_year['resultId'] * 100)
    sprint_by_year.columns = ['with_sprint', 'total_results', 'coverage_pct']
    # Only show years with some sprint data
    sprint_by_year_filtered = sprint_by_year[sprint_by_year['with_sprint'] > 0]
    print(sprint_by_year_filtered)

# Check driver_standings coverage by year
if 'driverStandingsId' in master.columns:
    print("\n\nDriver Standings Coverage by Year:")
    print("=" * 60)
    standings_by_year = master.groupby('year').agg({
        'driverStandingsId': lambda x: x.notna().sum(),
        'resultId': 'count'
    })
    standings_by_year['coverage_pct'] = (standings_by_year['driverStandingsId'] / standings_by_year['resultId'] * 100)
    standings_by_year.columns = ['with_standings', 'total_results', 'coverage_pct']
    # Show years with low coverage
    low_coverage = standings_by_year[standings_by_year['coverage_pct'] < 95]
    if len(low_coverage) > 0:
        print("Years with <95% coverage:")
        print(low_coverage)
    else:
        print("All years have >95% coverage")


**Expected Output:**
- **Qualifying**: Coverage should increase over time (better data in recent years), typically 85-95%
- **Sprint Results**: Should only appear from 2021+ (sprints introduced in 2021), ~2-3% overall
- **Driver Standings**: Should be >95% for all years (very high coverage)

**What to look for:**
- ✅ **Good signs**: 
  - Qualifying coverage consistent or improving over time
  - Sprint results only in 2021+
  - Standings consistently high
- ⚠️ **Warning signs**: 
  - Sudden drops in qualifying coverage (data gap)
  - Sprint results before 2021 (data quality issue)
  - Low standings coverage in any year


## 8. Summary Report

Generate a comprehensive summary report of all findings.


In [ ]:
# Generate comprehensive summary
# Check which variables exist from previous cells
duplicates_found = 0
if 'duplicates' in locals():
    duplicates_found = len(duplicates)

if 'missing_years' not in locals():
    expected_years = set(range(1994, 2025))
    actual_years = set(master['year'].unique())
    missing_years = sorted(list(expected_years - actual_years))
else:
    missing_years = sorted(missing_years) if missing_years else []

if 'year_coverage' not in locals():
    year_coverage = master.groupby('year').agg({'raceId': 'nunique'})
    year_coverage.columns = ['unique_races']

if 'merge_df' not in locals():
    merge_df = pd.DataFrame()

summary = {
    'dataset': 'master_races.csv',
    'verification_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'basic_stats': {
        'total_rows': len(master),
        'total_columns': len(master.columns),
        'date_range_start': str(master['date'].min()),
        'date_range_end': str(master['date'].max()),
        'year_range': f"{master['year'].min()}-{master['year'].max()}"
    },
    'uniqueness': {
        'unique_race_driver_pairs': len(master.groupby(['raceId', 'driverId'])),
        'duplicates_found': duplicates_found,
        'status': 'PASS' if duplicates_found == 0 else 'FAIL'
    },
    'temporal_coverage': {
        'expected_years': list(range(1994, 2025)),
        'actual_years': sorted(master['year'].unique().tolist()),
        'missing_years': missing_years,
        'avg_races_per_year': float(year_coverage['unique_races'].mean())
    },
    'merge_success_rates': merge_df.to_dict('records') if len(merge_df) > 0 else [],
    'data_quality': {
        'invalid_positions': int((master['positionOrder'] <= 0).sum()),
        'invalid_dates': int(master['date'].isnull().sum()),
        'year_date_mismatches': int((master['date'].dt.year != master['year']).sum()),
        'podium_rate': float(master['podium'].mean()) if 'podium' in master.columns else None
    }
}

# Print summary
print("Master Races Verification Summary")
print("=" * 60)
print(f"Dataset: {summary['dataset']}")
print(f"Verification Date: {summary['verification_date']}")
print(f"\nBasic Statistics:")
print(f"  Rows: {summary['basic_stats']['total_rows']:,}")
print(f"  Columns: {summary['basic_stats']['total_columns']}")
print(f"  Date Range: {summary['basic_stats']['date_range_start']} to {summary['basic_stats']['date_range_end']}")
print(f"  Year Range: {summary['basic_stats']['year_range']}")

print(f"\nUniqueness Check:")
print(f"  Unique (raceId, driverId) pairs: {summary['uniqueness']['unique_race_driver_pairs']:,}")
print(f"  Duplicates found: {summary['uniqueness']['duplicates_found']}")
print(f"  Status: {summary['uniqueness']['status']}")

print(f"\nTemporal Coverage:")
print(f"  Missing years: {missing_years if missing_years else 'None'}")
print(f"  Average races per year: {summary['temporal_coverage']['avg_races_per_year']:.1f}")

print(f"\nData Quality:")
print(f"  Invalid positions: {summary['data_quality']['invalid_positions']}")
print(f"  Invalid dates: {summary['data_quality']['invalid_dates']}")
print(f"  Year-date mismatches: {summary['data_quality']['year_date_mismatches']}")
if summary['data_quality']['podium_rate']:
    print(f"  Podium rate: {summary['data_quality']['podium_rate']*100:.2f}%")

print(f"\n✅ Dataset is ready for EDA and modeling!" if summary['uniqueness']['status'] == 'PASS' else "\n⚠️ Please review warnings above before proceeding.")

# Save summary to JSON
import json
summary_path = PROCESSED_ROOT / "master_races_verification_report.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"\nDetailed report saved to: {summary_path}")


**Expected Output:**
- Summary of all verification checks
- JSON report saved to `data/processed/master_races_verification_report.json`

**What to look for:**
- ✅ **Ready for next steps if**: 
  - No duplicates found
  - All critical checks pass
  - Merge success rates are acceptable
- ⚠️ **Review needed if**: 
  - Duplicates found
  - Critical columns missing
  - Data quality issues detected

**Next Steps:**
- If verification passes → Proceed to `03_exploratory_data_analysis.ipynb`
- If issues found → Review and fix in `02_data_combining.ipynb`, then re-run this verification
